In [ ]:
from flask import Flask, render_template, request
import snscrape.modules.twitter as sntwitter
import pandas as pd
import googletrans
import collections
from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline
#from happytransformer import HappyTextClassification
from textblob import TextBlob
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import seaborn as sns
sns.set()
%config InlineBackend.figure_format = 'retina'

app = Flask(__name__)


specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
def sentiment_hf(sentence):
    score = specific_model(sentence)
    return score

model = pipeline(model = "distilbert-base-uncased-finetuned-sst-2-english")
def sentiment_hf2(sentence):
    score = model(sentence)
    return score

def sentiment_textblob(row):
    classifier = TextBlob(row)
    polarity = classifier.sentiment.polarity
    if polarity < 0:
      label = "Negative"
    elif polarity == 0:
      label = "Neutral"
    else:
      label= "Positive"
    return {"polarity":polarity,"label":label}

#happy_tc = HappyTextClassification(model_type="DISTILBERT",  model_name="distilbert-base-uncased-finetuned-sst-2-english")
#def happy(sentence):
    #result = happy_tc.classify_text(sentence)
    #return {"score": result.score , "label": result.label}    


def sentiment_analyzer_scores(sentence):
    analyser = SentimentIntensityAnalyzer()
    score = analyser.polarity_scores(sentence)
    score = (score['compound'])
    if score >= 0.05 :
      return {"score": score , "label": "Positive"}
    elif score <= - 0.05 :
      return {"score": score , "label": "Negative"}
    else :
      return {"score": score , "label": "Neutral"}


@app.route("/")
def home():
    return render_template("keyword.html")

def fun(q,Keyword,Count):
  attributes_container = []
  for i,tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()):
      if Count != -1 and len(attributes_container) >= Count:
          break
      if Keyword in str(tweet.rawContent):
            attributes_container.append([tweet.user.username ,tweet.date, tweet.renderedContent])
  
  tweetKeyword = pd.DataFrame(attributes_container, columns=["user" ,"dateCreated","origTweet" ])
  tweetKeyword = tweetKeyword.drop_duplicates(keep='first')
  tweetKeyword['origTweetTranslated'] = tweetKeyword['origTweet'].apply(lambda x: Translator().translate(x, dest='en').text)
  tweetKeyword['Vader'] = tweetKeyword['origTweetTranslated'].apply(lambda x: sentiment_analyzer_scores(str(x)))
  tweetKeyword['HF1'] = tweetKeyword['origTweetTranslated'].apply(lambda x: sentiment_hf(str(x)))
  tweetKeyword['HF2'] = tweetKeyword['origTweetTranslated'].apply(lambda x: sentiment_hf2(str(x)))
  tweetKeyword['TextBlob'] = tweetKeyword['origTweetTranslated'].apply(lambda x: sentiment_textblob(str(x)))
  # tweetKeyword['Happy'] = tweetKeyword['origTweetTranslated'].apply(lambda x: happy(str(x)))
  tweetKeyword['FinalOrig'] = 'neg'
  for index, row in tweetKeyword.iterrows():
      arr = [str(row["Vader"]["label"]).lower()[0:3], str(row["HF1"][0]["label"]).lower()[0:3] , str(row["HF2"][0]["label"]).lower()[0:3] , str(row["TextBlob"]["label"]).lower()[0:3]]
      count = collections.Counter(arr)
      
      if count['neu']>= max(count['neg'] , count['pos']): 
        tweetKeyword.at[index , "FinalOrig"] = 'Neutral'
      elif count['pos'] >= max(count['neu'] , count['neg']):
        tweetKeyword.at[index , "FinalOrig"] = 'Positive'
      else:
        tweetKeyword.at[index , "FinalOrig"] = 'Negative'
  tweetKeyword.drop(columns=['Vader', 'HF1' , 'HF2' , 'TextBlob' ] , axis=1 , inplace = True)

  return tweetKeyword

@app.route('/keyword', methods=['POST'])
def res():
  Keyword = request.form["string-input"]
  Since = request.form["date1-input"]
  Until = request.form["date2-input"]
  q = ""  
  q += f"{Keyword}"
  if Until!='': 
    q+= f" until:{Until}" 
  if Since!='': 
    q += f" since:{Since}"
  Count = 50
  result = fun(q,Keyword,Count)
  _, axes = plt.subplots(nrows=1, ncols=1 ,figsize=(12, 4))
  sns.countplot(x='FinalOrig', data=result)
  img = BytesIO()
  plt.savefig(img , format='png')
  plt.close()
  img.seek(0)
  plot_url = base64.b64encode(img.getvalue())

  total = result['FinalOrig'].value_counts().values.sum()
  colors = sns.color_palette('muted')
  _ = plt.pie(result['FinalOrig'].value_counts().values, labels=result['FinalOrig'].value_counts().index, autopct='%.0f%%' , colors = colors)

  img2 = BytesIO()
  plt.savefig(img2 , format='png')
  plt.close()
  img2.seek(0)
  pie_url = base64.b64encode(img2.getvalue())

  return render_template('resultkey.html',tables=[result.to_html()], titles=result.columns.values , plot_url = plot_url.decode('utf-8') , pie_url = pie_url.decode('utf-8'))

In [ ]:
if __name__ == "__main__":
    app.run(port=5000,debug=False)